In [33]:
import pandas as pd
import dash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px

Task2

In [17]:
data_path = ['./data/daily_sales_data_0.csv', './data/daily_sales_data_1.csv', './data/daily_sales_data_2.csv']
# Load and process each file
dfs = []  # To hold data from each file
for file_path in data_path:
    # Load the CSV file
    df = pd.read_csv(file_path)
    
    # Filter rows where product is 'Pink Morsels'
    df = df[df['product'] == 'pink morsel']
    df['price'] = df['price'].str.replace('$', '').astype(float)
    # Calculate 'sales' as 'quantity' * 'price' and drop unnecessary columns
    df['sales'] = df['quantity'] * df['price']
    df = df[['sales', 'date', 'region']]
    
    # Append the processed DataFrame to the list
    dfs.append(df)

# Combine all DataFrames into one
combined_df = pd.concat(dfs, ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_df.to_csv('combined_sales_data.csv', index=False)

Task3

In [23]:
# Load the dataset
df = pd.read_csv('combined_sales_data.csv')
df['date'] = pd.to_datetime(df['date'])  # Ensure the date column is in datetime format
df.sort_values('date', inplace=True)  # Sort the data by date

# Initialize the Dash app
app = dash.Dash(__name__)

# Create a Plotly Express line chart
fig = px.line(df, x='date', y='sales', title='Pink Morsel Sales Over Time')
fig.update_layout(xaxis_title='Date', yaxis_title='Sales')

# Define the layout of the app
app.layout = html.Div(children=[
    html.H1(children='Soul Foods Sales Visualizer'),
    html.Div(children='''
        This line chart visualizes the sales of Pink Morsels over time.
    '''),
    dcc.Graph(
        id='sales-line-chart',
        figure=fig
    )
])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

Task4

In [34]:
# Load the dataset
df = pd.read_csv('combined_sales_data.csv')
df['date'] = pd.to_datetime(df['date'])  # Ensure the date column is in datetime format
df.sort_values('date', inplace=True)  # Sort the data by date

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the app with some basic styling
app.layout = html.Div(children=[
    html.H1(children='Soul Foods Sales Visualizer', style={'textAlign': 'center', 'color': '#007BFF'}),
    html.Div(children='''
        Visualize sales data for Pink Morsels by region.
    ''', style={'textAlign': 'center'}),
    dcc.RadioItems(
        id='region-selector',
        options=[
            {'label': 'North', 'value': 'north'},
            {'label': 'East', 'value': 'east'},
            {'label': 'South', 'value': 'south'},
            {'label': 'West', 'value': 'west'},
            {'label': 'All', 'value': 'all'}
        ],
        value='all',  # Default value
        style={'textAlign': 'center', 'margin': '20px'}
    ),
    dcc.Graph(id='sales-line-chart')
], style={'fontFamily': 'Arial, sans-serif'})

# Callback to update the line chart based on selected region
@app.callback(
    Output('sales-line-chart', 'figure'),
    Input('region-selector', 'value')
)
def update_chart(selected_region):
    if selected_region == 'all':
        filtered_df = df
    else:
        filtered_df = df[df['region'] == selected_region]
    
    # Create a line chart with the filtered data
    fig = px.line(filtered_df, x='date', y='sales', color='region', title='Pink Morsel Sales Over Time')
    fig.update_layout(xaxis_title='Date', yaxis_title='Sales', plot_bgcolor='white')
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

task5

In [35]:
!pip install dash[testing] pytest selenium

  Obtaining dependency information for pytest from https://files.pythonhosted.org/packages/2e/28/30125a808a2448d72fdba26d01ef2bec76a3c860c8694b636e6104e38713/pytest-8.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for selenium from https://files.pythonhosted.org/packages/97/e3/fd7272d6d2c49fd49a79a603cb28c8b5a71f8911861b4a0409b3c006a241/selenium-4.17.2-py3-none-any.whl.metadata
  Obtaining dependency information for beautifulsoup4>=4.8.2 from https://files.pythonhosted.org/packages/b1/fe/e8c672695b37eecc5cbf43e1d0638d88d66ba3a44c4d321c796f4e59167f/beautifulsoup4-4.12.3-py3-none-any.whl.metadata
  Obtaining dependency information for lxml>=4.6.2 from https://files.pythonhosted.org/packages/5b/d6/e794d996dec76b773691af78390fe2f419ab7cb5b78a4df982e21ae655b7/lxml-5.1.0-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/983.2 kB ? eta -:--:--
     --- ----------------------------------- 81.9/983.2 kB 2.3 MB/s eta 0:00:01
     --- ---------


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [36]:
import pytest
from selenium.webdriver.common.by import By
from dash.testing.application_runners import import_app


def test_header_presence(dash_duo):
    # Start the app
    app = import_app("your_dash_app_file_name")  # Replace with your Dash app's filename
    dash_duo.start_server(app)

    # Check if the header is present
    header = dash_duo.find_element("h1")
    assert header.text == "Soul Foods Sales Visualizer"

def test_visualisation_presence(dash_duo):
    app = import_app("your_dash_app_file_name")  # Replace with your Dash app's filename
    dash_duo.start_server(app)

    # Check if the visualization (graph) is present
    visualization = dash_duo.find_element("#sales-line-chart")
    assert visualization

def test_region_picker_presence(dash_duo):
    app = import_app("your_dash_app_file_name")  # Replace with your Dash app's filename
    dash_duo.start_server(app)

    # Check if the region picker (radio items) is present
    region_picker = dash_duo.find_element("#region-selector")
    assert region_picker
